In [1]:
%load_ext sql
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [2]:
%sql sqlite:///library.db

Connecting to 'sqlite:///library.db'

In [3]:
%%sql
CREATE TABLE Member (
    MemberID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Address TEXT,
    Email TEXT UNIQUE,
    Phone TEXT UNIQUE,
    MembershipType TEXT
);

CREATE TABLE Item (
    ItemID INTEGER PRIMARY KEY,
    Title TEXT NOT NULL,
    Genre TEXT,
    Type TEXT NOT NULL,
    Availability BOOLEAN NOT NULL
);

CREATE TABLE Book (
    ISBN TEXT PRIMARY KEY,
    Title TEXT NOT NULL,
    Author TEXT NOT NULL,
    Publisher TEXT,
    Year INTEGER,
    Edition TEXT,
    ItemID INTEGER UNIQUE,
    FOREIGN KEY (ItemID) REFERENCES Item(ItemID)
);

CREATE TABLE Multimedia (
    MediaID INTEGER PRIMARY KEY,
    Title TEXT NOT NULL,
    Artist TEXT,
    Type TEXT,
    Genre TEXT,
    ItemID INTEGER UNIQUE,
    FOREIGN KEY (ItemID) REFERENCES Item(ItemID)
);

CREATE TABLE Borrow (
    BorrowID INTEGER PRIMARY KEY,
    MemberID INTEGER,
    ItemID INTEGER,
    BorrowDate DATE NOT NULL,
    DueDate DATE NOT NULL,
    ReturnDate DATE,
    FOREIGN KEY (MemberID) REFERENCES Member(MemberID),
    FOREIGN KEY (ItemID) REFERENCES Item(ItemID)
);

CREATE TABLE Fine (
    FineID INTEGER PRIMARY KEY,
    MemberID INTEGER,
    Amount DECIMAL(10,2) NOT NULL,
    DueDate DATE NOT NULL,
    PaidStatus BOOLEAN NOT NULL,
    FOREIGN KEY (MemberID) REFERENCES Member(MemberID)
);

CREATE TABLE Event (
    EventID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Type TEXT,
    Date DATE NOT NULL,
    Time TEXT,
    AudienceType TEXT
);

CREATE TABLE Room (
    RoomID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Capacity INTEGER NOT NULL,
    Location TEXT
);

CREATE TABLE Personnel (
    PersonnelID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Role TEXT NOT NULL,
    ContactDetails TEXT
);

CREATE TABLE ItemRecord (
    RecordID INTEGER PRIMARY KEY,
    Title TEXT NOT NULL,
    ItemType TEXT,
    DateAdded DATE NOT NULL
);

CREATE TABLE Attends (
    MemberID INTEGER,
    EventID INTEGER,
    PRIMARY KEY (MemberID, EventID),
    FOREIGN KEY (MemberID) REFERENCES Member(MemberID),
    FOREIGN KEY (EventID) REFERENCES Event(EventID)
);

CREATE TABLE Manages (
    PersonnelID INTEGER,
    EventID INTEGER,
    PRIMARY KEY (PersonnelID, EventID),
    FOREIGN KEY (PersonnelID) REFERENCES Personnel(PersonnelID),
    FOREIGN KEY (EventID) REFERENCES Event(EventID)
);

CREATE TABLE Held (
    EventID INTEGER,
    RoomID INTEGER,
    PRIMARY KEY (EventID, RoomID),
    FOREIGN KEY (EventID) REFERENCES Event(EventID),
    FOREIGN KEY (RoomID) REFERENCES Room(RoomID)
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [4]:
%%sql
-- Prevent borrowing an item that is not available
CREATE TRIGGER PreventUnavailableBorrow
BEFORE INSERT ON Borrow
FOR EACH ROW
WHEN (SELECT Availability FROM Item WHERE ItemID = NEW.ItemID) = 0
BEGIN
    SELECT RAISE(ABORT, 'Item is not available for borrowing');
END;

-- Automatically update item availability when borrowed
CREATE TRIGGER UpdateItemAvailabilityBorrow
AFTER INSERT ON Borrow
FOR EACH ROW
BEGIN
    UPDATE Item SET Availability = 0 WHERE ItemID = NEW.ItemID;
END;

-- Automatically update item availability when returned
CREATE TRIGGER UpdateItemAvailabilityReturn
AFTER UPDATE ON Borrow
FOR EACH ROW
WHEN NEW.ReturnDate IS NOT NULL
BEGIN
    UPDATE Item SET Availability = 1 WHERE ItemID = NEW.ItemID;
END;

-- Prevent duplicate borrowing of the same item by the same member
CREATE TRIGGER PreventDuplicateBorrow
BEFORE INSERT ON Borrow
FOR EACH ROW
WHEN EXISTS (
    SELECT 1 FROM Borrow
    WHERE MemberID = NEW.MemberID AND ItemID = NEW.ItemID AND ReturnDate IS NULL
)
BEGIN
    SELECT RAISE(ABORT, 'Member has already borrowed this item and has not returned it');
END;

-- Automatically issue a fine for overdue items
CREATE TRIGGER IssueFineForOverdue
AFTER UPDATE ON Borrow
FOR EACH ROW
WHEN NEW.ReturnDate > NEW.DueDate
BEGIN
    INSERT INTO Fine (MemberID, Amount, DueDate, PaidStatus)
    VALUES (NEW.MemberID, 5.00 * (JULIANDAY(NEW.ReturnDate) - JULIANDAY(NEW.DueDate)), NEW.ReturnDate, 0);
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [5]:
%%sql
-- Insert sample Members
INSERT INTO Member (MemberID, Name, Address, Email, Phone, MembershipType) VALUES
(1, 'Alice Johnson', '123 Main St', 'alice@example.com', '1234567890', 'Regular'),
(2, 'Bob Smith', '456 Elm St', 'bob@example.com', '9876543210', 'Premium'),
(3, 'Charlie Brown', '789 Oak St', 'charlie@example.com', '5551234567', 'Regular');

-- Insert sample Items
INSERT INTO Item (ItemID, Title, Genre, Type, Availability) VALUES
(1, 'The Great Gatsby', 'Fiction', 'Book', 1),
(2, 'Interstellar', 'Sci-Fi', 'DVD', 1),
(3, 'Python Programming', 'Education', 'Book', 1),
(4, 'Bohemian Rhapsody', 'Music', 'CD', 1);

-- Insert sample Books
INSERT INTO Book (ISBN, Title, Author, Publisher, Year, Edition, ItemID) VALUES
('9780743273565', 'The Great Gatsby', 'F. Scott Fitzgerald', 'Scribner', 1925, '1st', 1),
('9780135166307', 'Python Programming', 'John Zelle', 'Franklin, Beedle & Associates', 2016, '3rd', 3);

-- Insert sample Multimedia
INSERT INTO Multimedia (MediaID, Title, Artist, Type, Genre, ItemID) VALUES
(1, 'Interstellar', 'Hans Zimmer', 'DVD', 'Sci-Fi', 2),
(2, 'Bohemian Rhapsody', 'Queen', 'CD', 'Rock', 4);

-- Insert sample Borrow records
INSERT INTO Borrow (BorrowID, MemberID, ItemID, BorrowDate, DueDate, ReturnDate) VALUES
(1, 1, 1, '2025-02-01', '2025-02-14', NULL),
(2, 2, 3, '2025-02-05', '2025-02-19', '2025-02-20'); -- Late return

-- Insert sample Fines
INSERT INTO Fine (FineID, MemberID, Amount, DueDate, PaidStatus) VALUES
(1, 2, 5.00, '2025-02-20', 0); -- Fine for late return

-- Insert sample Events
INSERT INTO Event (EventID, Name, Type, Date, Time, AudienceType) VALUES
(1, 'Book Club', 'Literary', '2025-03-01', '18:00', 'Adults'),
(2, 'Story Time for Kids', 'Children', '2025-03-05', '10:00', 'Children');

-- Insert sample Rooms
INSERT INTO Room (RoomID, Name, Capacity, Location) VALUES
(1, 'Conference Room A', 20, '1st Floor'),
(2, 'Children''s Room', 15, '2nd Floor'); 

-- Insert sample Personnel
INSERT INTO Personnel (PersonnelID, Name, Role, ContactDetails) VALUES
(1, 'David Green', 'Librarian', 'david@library.com'),
(2, 'Emma White', 'Event Coordinator', 'emma@library.com');

-- Insert sample Item Records
INSERT INTO ItemRecord (RecordID, Title, ItemType, DateAdded) VALUES
(1, 'The Great Gatsby', 'Book', '2024-10-15'),
(2, 'Interstellar', 'DVD', '2024-11-01');

-- Insert sample Attends relationships
INSERT INTO Attends (MemberID, EventID) VALUES
(1, 1),
(2, 2);

-- Insert sample Manages relationships
INSERT INTO Manages (PersonnelID, EventID) VALUES
(2, 1); -- Emma White manages the Book Club event

-- Insert sample Held relationships
INSERT INTO Held (EventID, RoomID) VALUES
(1, 1), -- Book Club in Conference Room A
(2, 2); -- Story Time in Children's Room



Running query in 'sqlite:///library.db'

3 rows affected.

4 rows affected.

2 rows affected.

2 rows affected.

...

2 rows affected.

2 rows affected.

1 rows affected.

2 rows affected.

++
||
++
++